In [1]:
import pandas as pd
import torch
from model.BSRModel import BSRModel


/home/wahoo/.pyenv/versions/3.13.2/envs/ootp-analytics/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
league =  'BZ'
season_start = 2016
season_end = 2018
ratings_type = "Standard"

In [3]:
bsrModel = BSRModel(league, season_start, season_end, ratings_type)

CUDA is available! Using GPU.


In [4]:
bsrModel.load_data(400)

(296, 12)
(594, 12)
(890, 12)
Empty DataFrame
Columns: [WT, SPE, SR, STE, RUN, BABIP, GAP, K's, BFH, GBT, FBT, BSR600]
Index: []


In [5]:
epoch_count, loss = bsrModel.train(18000)
print(f"Epoch [{epoch_count}/{50000}], Loss: {loss:.7f}")

test_loss = bsrModel.evaluate()
print(f"Test Error: {test_loss:.7f}")

#feature_importance = bsrModel.feature_importance()
#print(feature_importance)

bsrModel.save_model()


Epoch [2000/18000], Best Loss: 0.0671958, Best Eval: 10000.0000000
Epoch [4000/18000], Best Loss: 0.0455712, Best Eval: 10000.0000000
Epoch [6000/18000], Best Loss: 0.0374032, Best Eval: 10000.0000000
Epoch [8000/18000], Best Loss: 0.0305658, Best Eval: 10000.0000000
Epoch [10000/18000], Best Loss: 0.0305658, Best Eval: 10000.0000000
Epoch [12000/18000], Best Loss: 0.0288379, Best Eval: 10000.0000000
Epoch [14000/18000], Best Loss: 0.0288379, Best Eval: 10000.0000000
Epoch [16000/18000], Best Loss: 0.0243330, Best Eval: 10000.0000000
Epoch [18000/18000], Best Loss: 0.0213613, Best Eval: 10000.0000000
Epoch [17999/50000], Loss: 0.0467392
Test Error: 2.2890306
